In [131]:
import requests
import requests_cache
from bs4 import BeautifulSoup
# pip install lxml
import pickle
import random
from collections import defaultdict
import json

# debug
import sys

## Graph des directeurs de thèses --> thésards

Avec les données du site theses.fr, et l'api:
http://documentation.abes.fr/aidethesesfr/accueil/index.html#RecupererDonneesPagePersonne

info sur les Parser XML de BeautifulSoup: https://www.crummy.com/software/BeautifulSoup/bs4/doc/#installing-a-parser


- Les info sur les thèses sont récupérées avec des requètes sur une personne.
    - liste des thèses (la sienne, en tant que directeur, que rapporteur, et membre du jury)
    

puis sur les personnes connexes. 



In [132]:
def parse_foaf(tag):
    try:
        name = tag.text
        person_id = tag.findChild().attrs['rdf:about']
    except KeyError:
        # print(tag)
        name, person_id = (tag.text, None)
    return (name, person_id)


def short_id(auth, year):
    """ Robust id for people 
    """
    # th:   (('Name Name', 'http://www.theses.fr/0123456789/id'), 1296)
    name = auth[0]
    initials = ''.join( [u[0] for u in name.split()] )
    idx = auth[1].split('/')[-2]
    return initials+'_'+str(year)+'_'+idx

requests_cache.install_cache('thesis_cache')

def query_someone(person_id):
    ''' Request info about the person
        returns the person info (name and id) and list of related thesis
    '''
    url = person_id.replace('/id', '.xml')

    r = requests.get(url)
    
    soup = BeautifulSoup(r.text, 'lxml') # Set the parser
    
    # Person info:
    agent = soup.find('foaf:agent')
    person_id = agent.attrs['rdf:about']
    name = agent.find('foaf:name').text
    person = (name, person_id)
    
    # Thesis list:
    linked_thesis = []
    for th in soup.find_all('bibo:thesis'):
        info = dict()
        info['title'] = th.find('dc:title').text
        info['author'] = parse_foaf( th.find('marcrel:aut') )
        info['id'] = th.attrs['rdf:about']
        info['url'] = th.find('dc:identifier').text if th.find('dc:identifier') else ''
        try:
            info['year'] = int(th.find('dc:date').text)
        except ValueError:
            info['year'] = 0
        info['directors'] = [parse_foaf(ths) for ths in th.find_all('marcrel:ths')]
        info['doctoral_school'] = parse_foaf(th.find('dcterms:contributor')) if th.find('dcterms:contributor') else ()
        info['univ'] = parse_foaf(th.find('marcrel:dgg'))
        info['short_id'] = short_id(info['author'], info['year'])
        
        linked_thesis.append(info)

    return person, linked_thesis

In [133]:
def is_valid_id(foaf_id):
    return foaf_id.endswith('/id') and not foaf_id.endswith('//id')

In [134]:
# Objet qui stocke les données explorées
# et les requetes suivante à faire
class Blob():
    def __init__(self, seed_id):
        self.already_asked = set()
        self.to_search = set([seed_id, ])
        self.thesis = dict()
        self.seed = seed_id
        self.chains = []
        self.nbr_gen = 0
        self.name = seed_id.split('/')[-2]
        
    def grow(self):
        person_id = self.to_search.pop()
       
        full_id, results = query_someone(person_id)
        print('\r', full_id[0], '(%i theses)'%len(results), end=' '*20)
        self.already_asked.add(person_id)
        self.nbr_gen += 1
        
        # default value
        own_thesis = {'author':full_id,
                      'year':0,
                      'id':None,
                      'directors':[],
                      'short_id':short_id(full_id, 0)} 
        
        # Sort the thesis (own, as a director, other)
        a_dirige = []
        related = []     
        for th in results:
            if full_id == th['author']:
                own_thesis = th
            elif full_id in (d for d in th['directors']):
                a_dirige.append(th)
            else:
                related.append(th)
        
        a_dirige.sort(key=lambda x:x['year'])

        # Update new persons to search:
        co_directors = {d for th in a_dirige+[own_thesis, ]
                          for d in th['directors'] if d!=full_id}
        students = {th['author'] for th in a_dirige}

        new_person = [p[1] for p in co_directors | students
                      if is_valid_id(p[1]) and p[1] not in self.already_asked]

        self.to_search.update(new_person)
        
        # Save thesis:
        self.thesis[own_thesis['short_id']] = own_thesis
        self.thesis.update({th['short_id']:th for th in a_dirige})
        
        # Build 'chain' (for subway graph)
        if len(a_dirige) > 0:
            chain = [own_thesis['short_id'], ] + [th['short_id'] for th in a_dirige]
            self.chains.append(chain)
            
        
    def save(self, name='blob.p'):
        pickle.dump(self, open( name, "wb" ))
        print("saved")
        
    def print_info(self):
        print('nbr thesis:', len(blob.thesis))
        print('nbr to search:', len(blob.to_search))
        print(self.seed, '+', self.nbr_gen, 'generations')

In [135]:
# Colors
tableau20 = ['#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c',
             '#98df8a', '#d62728', '#ff9896', '#9467bd', '#c5b0d5',
             '#8c564b', '#c49c94', '#e377c2', '#f7b6d2', '#7f7f7f',
             '#c7c7c7', '#bcbd22', '#dbdb8d', '#17becf', '#9edae5']
# http://vis.stanford.edu/color-names/analyzer/

def get_color(string):
    k = hash(string) % len(tableau20)
    return tableau20[k]

In [262]:
# Get a random seed
seedlist_file = "data/seedlist.pick"
try:
    seedlist = pickle.load( open( seedlist_file, "rb" ) )
    print('seedlist loaded:', len(seedlist), "seeds")
except FileNotFoundError:
    seedlist = ['http://www.theses.fr/068670648/id', ]
    pickle.dump(seedlist, open( seedlist_file, "wb" ))
    seedlist = pickle.load( open( seedlist_file, "rb" ) )
    print('seedlist loaded:', len(seedlist), "seeds")
    
seed_id = random.choice(seedlist)
print(seed_id)

# New graph
blob = Blob(seed_id)

seedlist loaded: 51 seeds
http://www.theses.fr/162509553/id


In [263]:
blob.grow()

 Cynthia Joseph Eid (2 theses)                    

In [264]:
# Grow
for _ in range(40):
    blob.grow()

print('\n')
blob.print_info()

 Bernard Bonnetot (3 theses)                                     

nbr thesis: 129
nbr to search: 140
http://www.theses.fr/162509553/id + 41 generations


In [265]:
# Export the graph 
filename_prefix = 'data/'+blob.name+'_'+str(len(blob.thesis))
print(filename_prefix)

# Chains (for the layout)
chs = sorted(blob.chains, key=len, reverse=True)
print(len(chs), "chains")

with open(filename_prefix + '_chains.json', 'w') as outfile:
    json.dump(chs, outfile)

data/162509553_129
19 chains


In [11]:
# Extract chain info and update
chain_info = dict()
for chain in blob.chains:
    name = blob.thesis[chain[0]]['author'][0]
    color = get_color(name)
    chain_info[chain[0]] = {'length': len(chain),
                            'color': color,
                            'name': name,
                            'year':blob.thesis[chain[0]]['year']}
    for n in chain:
        blob.thesis[n]['color'] = color
        
        if blob.thesis[n]['author'][1] in blob.already_asked:
            blob.thesis[n]['searched'] = 1
        else:
            blob.thesis[n]['searched'] = 0
            
    blob.thesis[chain[0]]['nbr_thesis'] = len(chain)
    #blob.thesis[chain[0]]['color'] = red
    
# Export the info
with open(filename_prefix + '_chaininfo.json', 'w') as outfile:
    json.dump(chain_info, outfile)

# Export the thesis info
with open(filename_prefix + '_thesis.json', 'w') as outfile:
    json.dump(blob.thesis, outfile)
    
print(len(blob.thesis), 'thesis')

99 thesis


In [10]:
# Save
#blobname = 'data/grosblob6.p'
#blob.save(blobname)

# local growth

In [212]:
def sort_thesis(full_id, results):
    # default value
    own_thesis = {'author':full_id,
                  'year':0,
                  'id':None,
                  'directors':[],
                  'short_id':short_id(full_id, 0)} 

    # Sort the thesis (own, as a director, other)
    a_dirige = []
    related = []     
    for th in results:
        if full_id == th['author']:
            own_thesis = th
        elif full_id in (d for d in th['directors']):
            a_dirige.append(th)
        else:
            related.append(th)
    
    a_dirige.sort(key=lambda x:x['year'])

    return own_thesis, a_dirige, related

In [246]:
seed_id = random.choice(seedlist)
print(seed_id)

http://www.theses.fr/068670648/id


('Arnaud Brioude', 'http://www.theses.fr/068670648/id') 25 11


In [259]:
# gen zero
full_id, results = query_someone(seed_id)
own_thesis, a_dirige, related = sort_thesis(full_id, results)

print(full_id, len(results), len(a_dirige))

thesis = dict()
chains = []

# save thesis
thesis[own_thesis['short_id']] = own_thesis
for th in a_dirige:
    thesis[th['short_id']] = th

# Build 'chain' (for subway graph)
if len(a_dirige) > 0:
    chain = [own_thesis['short_id'], ] + [th['short_id'] for th in a_dirige]
    chains.append(chain)
else:
    print('no thesis')

('Arnaud Brioude', 'http://www.theses.fr/068670648/id') 25 11


In [260]:
print(len(thesis))

12


In [261]:
second_gen = {th['author'][1] for th in a_dirige if is_valid_id(th['author'][1])}
print('second gen', len(second_gen))

third_gen = set()
for id_to_search in second_gen:
    full_id, results = query_someone(id_to_search)
    own_thesis, a_dirige, related = sort_thesis(full_id, results)
    print(len(a_dirige))
    # save thesis
    thesis[own_thesis['short_id']] = own_thesis
    for th in a_dirige:
        thesis[th['short_id']] = th
        
    # Build 'chain' (for subway graph)
    if len(a_dirige) > 0:
        chain = [own_thesis['short_id'], ] + [th['short_id'] for th in a_dirige]
        third_gen.update({th['author'][1] for th in a_dirige if is_valid_id(th['author'][1])})
        chains.append(chain)   

second gen 11
0
0
0
0
0
0
0
0
0
0
0


In [257]:
print(len(thesis))

12


In [250]:
accepted_third_gen_thesis = dict()
for id_to_search in third_gen:
    full_id, results = query_someone(id_to_search)
    own_thesis, a_dirige, related = sort_thesis(full_id, results)
    
    th_already_known = len([th for th in a_dirige if th['short_id'] in thesis])
    if own_thesis['short_id'] in thesis:
        th_already_known += 1
        
    if th_already_known <= 1:
        continue
    else:
        # save thesis
        accepted_third_gen_thesis[own_thesis['short_id']] = own_thesis
        for th in a_dirige:
            accepted_third_gen_thesis[th['short_id']] = th

        # Build 'chain' (for subway graph)
        if len(a_dirige) > 0:
            chain = [own_thesis['short_id'], ] + [th['short_id'] for th in a_dirige]
            chains.append(chain)  

In [251]:
accepted_third_gen_thesis

{}

In [225]:
th_already_known

1

In [253]:
len(thesis)

12